# PAREI NA AULA 450 - PROJETO 2 - AUTOMAÇÕES WEB - [Acesso aqui!](https://t.me/c/2104024236/453)

# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail crowtler@proton.me)

### O passo a passo para o projeto
1. Criar um navegador
2. Importar/vizualizar a base de dados
3. Para cada item dentro da nossa base de dados, para cada produto
   1. Procurar esse produto no google shoping
      1. Verificar se algum dos produtos do google shoping está dentro da minha faixa de preço
   2. Procurar esse produto no buscapé
      1. Verificar se algum dos produtos do buscapé está dentro da minha faixa de preço
4. Salvar as ofertas boas em um dataframe (tabela)
5. Exportar para um excel
6. Enviar por e-mail o resultado da tabela.

0. Importações
   

In [1]:
import pandas as pd
import win32com.client as win32
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

1. Criação do navegador.

In [2]:
buscas = pd.read_excel('buscas.xlsx')
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument(r'--user-data-dir=C:\Users\albin\AppData\Local\Google\Chrome\User Data\Selenium')
nav = webdriver.Chrome(service=service, options=options)

2. Funções de busca

In [3]:
def verificar_existencia_termos_banidos(lista_termos_banidos, name):
    tem_termos_banidos = False
    for termo in lista_termos_banidos:
        if termo in name:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_existencia_todos_termos_produtos(lista_termos_nomes, name):
    tem_todos_termos_produtos = True
    for termo in lista_termos_nomes:
        if termo not in name:
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos

def busca_google_shoping(nav, produto, termos_banidos, preco_minimo, preco_maximo):

    produto = produto.lower()

    termos_banidos = termos_banidos.lower()

    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nomes = produto.split(" ")

    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)


    link = "https://www.google.com/"

    nav.get(link)


    search_field = nav.find_element(By.CLASS_NAME, "gLFyf")

    search_field.click()

    search_field.send_keys(produto)

    search_field.send_keys(Keys.ENTER)


    shoping = nav.find_element(
        By.XPATH, '//*[@id="hdtb-sc"]/div/div/div[1]/div/div[2]/a'
    )

    shoping.click()


    nav.execute_script("window.scrollTo(0, 700)")


    list_results = nav.find_elements(By.CLASS_NAME, "i0X6df")


    for result in list_results:

        name = result.find_element(By.CLASS_NAME, "tAxDx").text

        name = name.lower()


        tem_termos_banidos = verificar_existencia_termos_banidos(
            lista_termos_banidos, name
        )
        tem_todos_termos_produtos = verificar_existencia_todos_termos_produtos(
            lista_termos_nomes, name
        )


        if not tem_termos_banidos and tem_todos_termos_produtos:

            price = result.find_element(By.CLASS_NAME, "a8Pemb").text
            price = (
                price.replace("R$", "")
                .replace(" ", "")
                .replace(".", "")
                .replace(",", ".")
            )
            if price.isdigit():
                price = float(price)
            else:
                price = price.replace('+impostos', '')
                price = float(price)


            if preco_minimo <= price <= preco_maximo:

                link_reference = result.find_element(By.CLASS_NAME, "bONr3b")

                link_parent = link_reference.find_element(By.XPATH, "..")

                link = link_parent.get_attribute("href")


                lista_ofertas.append((name, price, link))


    return lista_ofertas

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nomes = produto.split(" ")
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    nav.get("https://www.buscape.com.br/")
    nav.find_element(
        By.XPATH,
        '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input',
    ).send_keys(produto, Keys.ENTER)

    while len(nav.find_elements(By.CLASS_NAME, "Select_Select__1HNob")) < 1:
        time.sleep(1)
    nav.execute_script("window.scrollTo(0, 250)")

    lista_resultados = nav.find_elements(
        By.CLASS_NAME, "ProductCard_ProductCard_Inner__gapsh"
    )

    for result in lista_resultados:
        price = result.find_element(By.CLASS_NAME, "Text_MobileHeadingS__HEz7L").text
        name = result.find_element(
            By.CLASS_NAME, "ProductCard_ProductCard_Name__U_mUQ"
        ).text
        name = name.lower()
        link = result.get_attribute("href")

        tem_termos_banidos = verificar_existencia_termos_banidos(
            lista_termos_banidos, name
        )
        tem_todos_termos_produtos = verificar_existencia_todos_termos_produtos(
            lista_termos_nomes, name
        )

        if not tem_termos_banidos and tem_todos_termos_produtos:
            price = (
                price.replace("R$", "")
                .replace(" ", "")
                .replace(".", "")
                .replace(",", ".")
            )
            price = float(price)

            if preco_minimo <= price <= preco_maximo:
                lista_ofertas.append((name, price, link))

    return lista_ofertas

3. Tabela de ofertas

In [4]:
tabela_ofertas = pd.DataFrame()

for linha in buscas.index:
    produtos_pesquisa = buscas['Nome']
    preco_mínimo_comparacao = buscas['Preço mínimo']
    produto = buscas.loc[linha, 'Nome']
    termos_banidos = buscas.loc[linha, 'Termos banidos']
    preco_minimo = buscas.loc[linha, 'Preço mínimo']
    preco_maximo = buscas.loc[linha, 'Preço máximo']

    lista_ofertas_google_shoping = busca_google_shoping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shoping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shoping, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None

display(tabela_ofertas)

,Produto,Preço,Link
0,"iphone 12 apple 64gb preto tela 6,1” 12mp ios",3110.00,https://www.google.com/url?url=https://shoppin...
1,iphone 12 64gb azul - excelente,3190.00,https://www.google.com/url?url=https://www.cdk...
0,smartphone apple iphone 12 vermelho 64gb câmer...,3134.99,https://www.buscape.com.br/celular/smartphone-...
0,placa de vídeo msi rtx 3060ti twin fan oc edit...,4299.00,https://www.google.com/url?url=https://www.inf...
0,"notebook acer aspire 3 a315-510p-34xc, intel c...",2599.99,https://www.google.com/url?url=https://www.kab...
1,notebook acer aspire 5 a514-54-324n - intel co...,2676.51,https://www.google.com/url?url=https://www.loj...
2,"usado - notebook acer aspire 5, intel core i3-...",2307.60,https://www.google.com/url?url=https://www.kab...
3,notebook acer aspire 3 a315-58-38sd - intel co...,2314.75,https://www.google.com/url?url=https://www.kab...
0,notebook acer aspire 3 a315-510p-34xc intel co...,2217.51,https://www.buscape.com.br/notebook/notebook-a...
1,notebook acer aspire 3 a315-58-31uy intel core...,2384.10,https://www.buscape.com.br/notebook/notebook-a...


4. Exportando para o Excel

In [5]:
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

5. Enviando o e-mail

In [6]:
outlook = win32.Dispatch('outlook.application')

if len(tabela_ofertas) > 0:
    mail = outlook.CreateItem(0)
    mail.To = 'crowtler@proton.me'
    mail.Subject = "Produto(s) encontrado(s) na faixa de preço desejada."
    mail.HTMLBody = f"""
    <p>Prezado,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. </p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att. Albino Marques</p>
    """
    mail.Send()
    nav.quit()